# Hands-On Machine Learning

## Session 4: Evaluation, Validation and Model Assessment
by Laxmi Gupta, Christoph Haarburger

### Goals of this Session

In this session you will...
* learn to assess a model's performance
* get to know the most common evaluation measures
* learn how to tune a model's hyperparameters

### Iris Dataset

You already know the Iris dataset from you first session. Anyway, here's a quick recap: The dataset consists of three species (=classes) of the Iris plant (Setosa, Versicolor and Virginica). For each sample, four features have been measured (sepal length, sepal width, petal length, petal width). 

The goal ist to train a model that can classify a given sample into one of the three species. Based on your knowledge of classifiers from the 1st session, today we'll focus more on the evaluation of performance and tuning of hyperparameters rather than the classifier itself.

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns
from evaluation import load_iris
data = load_iris()

Before training any kind of model it is always a good idea to make ourselves familiar with the raw data and explore basic properties of the dataset. The dataset has already been loaded as a `pandas.DataFrame` which is a very handy data type for our exploration.

In [ ]:
data[:15]

You my find it interesting to look at the raw numbers, but with our goal of building a classifier in mind we particularly want to explore statistical properties of the dataset.

In [ ]:
data.describe()

**Task**: Find out if the prevalence of the three classes is equal!

In [ ]:
import numpy as np
y = data['species'].values # turn pandas.DataFrame into np.ndarray




We're lucky, all classes are represented in equal ratios 😅.

If the number of features is relatively small as in our case, it is also a good idea to plot features pairswise against each other and inspect the feature scores for all classes as histograms.

In [ ]:
_ = sns.pairplot(data, hue='species')

**Task:** Can the classification problem be solved by a linear classifier?

**Answer:** 



Now that we have gained some intuitution on the problem to be solved und the underlying data, let's get to what we are actually up to: Classification! As a first shot, it is always a good idea to utilize a classifier that is easy to understand and easy to interpret. K-Nearest-Neighbors does the job...

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

X = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy() # turn pandas.DataFrame into np.ndarray

model = KNN(n_neighbors=2)
model.fit(X, y)
y_pred = model.predict(X)

**Task:** To interpret the result, implement the accuracy measure and calculate the accuracy for our predictions. Accuracy is defined as $\frac{\mathrm{true}_\mathrm{positive} + \mathrm{true}_\mathrm{negative}}{\mathrm{positive} + \mathrm{negative}}$.

In [ ]:
def accuracy_score(y_true, y_pred):




**Task:** Why is the classification accuracy nearly perfect? 


**Answer:**



## Holdout Sets

A much better way to evaluate performance is to split all available data into a training and test set, where the training set is used for training the model's parameters and the test set is utilized for assessing performance. 

**Task:** Implement the same pipeline as in the last code cell but with a separate training and test set (50/50)!

In [ ]:
from sklearn.model_selection import train_test_split




That looks a lot more realistic! 

**Task:** When using `train_test_split`, the samples are *randomly* drawn to one of the sets. What happens, if we use a different split? Determine the 95% confidence intervals for random 100 splits. *Hint:* `np.percentile`

**Task:** What properties should a 'good' split have with respect to the dataset?

**Answer:**



## Cross Validation

One disadvantage of the holdout set method is that half the dataset does not contribute to the training of the model. Especially if the initial set of training data is small this is suboptimal since statistical learning methods tend to perform worse when trained on fewer observations.

In cross validation we can overcome this by partitioning the data set in `k` "folds" of approximately equal size. The first fold is used as a test set and the model is trained on the remaining `k` - 1 folds. The test error is then co puted on the test set and the process is repeated `k` times, resulting in `k` estimates of the test error.

**Task:** Implement the `CrossValidation` class as sketched in the following cell! 
Hint: This is advanced task that is not mandatory to be solved.

**Task:** Use the `CrossValidation` class to evaluate the test error of the same KNN model as above but with 10 fold cross validation. What's the mean test error? If you did not solve the previous task on your own, you can use `KFold` from `scikit-learn`.

**Task:** Why did the accuracy improve compared to the holdout split? What's a disadvantage of the cross validation approach?

**Answer:**



## Hyperparameter Optimization

KNN is well known for being easy to use and interpret, howerever to achieve top performance it is not the best choice. In the first Session you got to know the Support Vector Machine (SVM) classifier. In contrast to KNN it has quite a few hyperparameters that need to optimized to achieve top performance. In the next few steps we will optimize SVM hyperparameters by a grid search.

SVMs are not scale invariant, i.e. all features need to be scaled to the same range, for example [-1, 1] or [0, 1]. The most common approach to this is the subtraction of the mean and division by the standard deviation of each feature.

It is one of the most common errors in machine learning to carry out scaling on the whole dataset rather than on training- and test set independently. If this is not considered, information about the distribution of the test set (mean and standard deviation) leaks into the training set and the cross validation is flawed.

A very handy way to overcome this is using the `sklearn.pipeline.Pipeline` class for combining consecutive processing pipelines such as scaling and classification.

**Task:** Combine an SVM classifier (default parameters) and a `StandardScaler` in a `Pipeline`. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC




If you now call `pipe.fit(X_train, y_train)`, the pipeline calculcates mean and standard deviation based on the training data, applies the scaling and trains the SVM.

It's time to tune the SVM's hyperparameters, namely the `C` parameter, the type of kernel and the `gamma` parameter.
The naive approach to hyperparameter tuning lies in exhaustively testing all possible combinations of parameters, which is called grid search. 

**Task:** Implement a parameter grid for our three parameters to tune and perform the grid search in a 10-fold cross validation!

**Hint:** Keep in mind that exhaustive grid search results in high computation times. Use parallelization with the `n_jobs` argument and keep track of time with the `%time` magic command.

In [ ]:
from sklearn.model_selection import GridSearchCV




Take a closer look at the grid search results. What mean test score can you achieve?

In [ ]:
from evaluation import show_grid_search_results
show_grid_search_results(gcv)[:5]

As the number of hyperparameter is usually much higher than here, a naive grid search is often not feasible in practice. A good alternative lies in a randomized search, in which the parameter space is sampled coarsely.

**Task:** Implement the same workflow as above withe the same parameter range but using `RandomizedSarchCV`. Can you achieve comparable performance in shorter time?

You probably found out that it is possible to achieve literally the same mean test score using a randomizes search in just the fraction of the time needed for al full grid search.

**Task:** Is the mean test score a good estimation of the performance we can expect on unseen data?

**Answer:**



## Evaluation Measures

To make this more interesting, the iris dataset was slightly modified to better resemble real-world datasets. The following modified Iris dataset is smaller, there are only two classes present and the prevalence for the two classes is imbalanced.

In [ ]:
data = load_iris(imbalanced_binary=True)
_ = sns.pairplot(data, hue='species', vars=data.columns[:-1])

**Task:** Solve the binary classification problem by a KNN classifier, optimize the number of neighbors by 5 fold cross validation and evaluate accuracy on the test set using the best `n_neighbors`. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy(), data[['species']].values.flatten(), test_size=0.5, random_state=1)




Not too bad. You probably achieved an accuracy near 90%. After this first result we should take a more detailed look at the performance using a confusion matrix, i.e. find out the number of true positives, true, negatives, false positives and false negatives.

Apparently, we only had four positive samples in the test set, out of which one half was classified incorrectly.

**Task:** What do you conclude about accuracy as a metric? Whats the accuracy of just labelling all samplese as negative?

** Solution:**



### ROC Curves
A better way to assess classification performance for imbalanced problems lies in looking at the *true positive rate* and *false positive rate* (also known as sensitivity and specifity).

These two can even be plotted against each other, which results in a receiver operating curve (ROC). The higher the area under the ROC curve (AUC) the better the classifier.

This is especially useful in medical image processing, a context in which we usually have data from a population out of which a small fraction has a particular disease that need to be detected.

**Task:** Whats are the *true positive rate*, *false positive rate* and *AUC* of our classifier? Plot the ROC curve!

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve




## Yayy, high accuracy!... Just by chance?

**Task:** What if we were to consider only two features for classification, which ones would you choose? Would the accuracy be 'significantly' different depending on the choice of features?  

**Answer:**



We will use the scipy.stats module for simple statistical tests.

In [ ]:
from scipy import stats

## Student's t-test

Let us consider two feature groups and see if we get similar accuracies or not:
    1. only sepal length and petal length
    2. only sepal width and petal width

In [ ]:
data = load_iris()
X = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()
X_length = data[['petal_length', 'sepal_length']].to_numpy() # Considering only lengths 
X_width = data[['sepal_width', 'petal_width']].to_numpy() # Considering only widths

y = data['species'].values

scores = np.zeros(100)
scores_length = np.zeros(100)
scores_width = np.zeros(100)

# Calculate accuracy when considering all the features
for i in range(100):    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=i, test_size=0.5)

    model = KNN(n_neighbors=2)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    scores[i] = accuracy_score(y_test, y_pred)
print('Mean accuracy considering all the features = {:2.3}'.format(scores.mean()))

# Calculate accuracy when considering only lengths
for i in range(100):    
    X_length_train, X_length_test, y_length_train, y_length_test = train_test_split(X_length, y, random_state=i, test_size=0.5)
    
    model = KNN(n_neighbors=2)
    model.fit(X_length_train, y_length_train)
    
    y_length_pred = model.predict(X_length_test)
    scores_length[i] = accuracy_score(y_length_test, y_length_pred)
print('Mean accuracy considering petal_length and sepal_length only = {:2.3}'.format(scores_length.mean()))

# Calculate accuracy when considering only widths
for i in range(100):    
    X_width_train, X_width_test, y_width_train, y_width_test = train_test_split(X_width, y, random_state=i, test_size=0.5)
    
    model = KNN(n_neighbors=2)
    model.fit(X_width_train, y_width_train)
    
    y_width_pred = model.predict(X_width_test)
    scores_width[i] = accuracy_score(y_width_test, y_width_pred)
print('Mean accuracy considering petal_width and sepal_width only = {:2.3}'.format(scores_width.mean()))

### 2-sample t-test: testing for difference across populations

The means of both the groups are different. To test if this is significant, we perform a 2-sample t-test with scipy.ttest_ind()

In [ ]:
stats.ttest_ind(scores_length, scores_width)

If the observed p-value is smaller than the threshold value (typically 0.01, 0.05, 0.1), we reject the null hypothesis of equal means. In this case, however, we fail to find statistical difference.

**Task:** Are the groups X_petal and X_sepal significantly different from X?

*Conclusion*

    X_petal is significantly different from X 
    X_sepal is significantly different from X 

### Paired t-test: repeated measurements

Note that the measurements for petal and sepal are made on the same flowers! In such a case where each subject is measured twice, we can use a paired sample t-test.

In [ ]:
stats.ttest_rel(scores_length, scores_width)

Another thing we overlooked is the fact that t-tests assume a gaussian distribution of the data. But is this true for our data? Let's investigate:

In [ ]:
# Plot the histograms to see the data distribution

bins = 25
data_x = (scores, scores_length, scores_width)
x_labels = ('scores', 'scores_length', 'scores_width')
xlims = [[0.85, 1]]*3
ylims = [[0, 30]]*3

for j, (x, xlim, ylim, x_label) in enumerate(zip(data_x, xlims, ylims, x_labels)):
    ax = plt.subplot(1, 3, j+1)
    ax.hist(np.reshape(x,(-1,1)), bins = bins)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xlabel(x_label)
    
ax = plt.subplot(1,3,1)
ax.set_ylabel('frequency');

Oops! Our data does not have a gaussian distibution! To take this into account, we must use Wilcoxon signed-rank test, with the help of scipy.stats.wilcoxon().

**Task:** Calculate the wilcoxon signed-rank test for the groups above. Draw a conclusion!

pvalue > 0.1 Hence we accept the Null Hypothesis of equal means, which concludes that we fail to find significant differences at p=0.1 

**Note:** The corresponding test for non-paired case is the Mann-Whitney U test (scipy.stats.mannwhitneyu)

## Discussion
In case of unbalanced data, discuss the pros and cons of different metrics for evaluating a classifier.


How do you ensure you’re not overfitting with a model?

### Feedback

That's it, we're done 👏🏼🍻

If you have any suggestions on how we could improve this session, please let us know in the following cell. What did you particularly like or dislike? Did you miss any contents?